In [1]:
print("STARTO")
import time
import os
start_time = time.time()

using_gpu = True
if not using_gpu:
    os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
    os.environ["CUDA_VISIBLE_DEVICES"] = ""

from lib.detector.YOLO.train.trainer import Trainer
from lib.detector.YOLO import create_detector_model_from_trained
from lib.detector.YOLO import create_training_model_from_detector
from lib.detector.YOLO import Yolo

STARTO


Using TensorFlow backend.


In [2]:
from contextlib import contextmanager

@contextmanager
def timer(title):
    t0 = time.time()
    yield
    print("{} - done in {:.0f}s".format(title, time.time() - t0))

In [3]:
from keras.models import load_model
pretrained = load_model('./v1.h5')

/src/keras/engine/saving.py:269: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [4]:
pretrained.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 608, 608, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 608, 608, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 608, 608, 32) 128         conv2d_1[0][0]                   
__________________________________________________________________________________________________
leaky_re_lu_1 (LeakyReLU)       (None, 608, 608, 32) 0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
max_poolin

In [5]:
# pretrained_model = create_training_model_from_detector(pretrained)
pretrained_model = pretrained
for i in range(len(pretrained_model.layers)):
    pretrained_model.layers[i].trainable = False

In [6]:
pretrained_model.layers[-1].trainable = True
# pretrained_model.layers[-4].trainable = True

In [7]:
labels = ['bicycle','bus','car','motorbike']

In [8]:
import numpy as np
import json
with open('ua_voc_4.json', 'r') as fp:
    data = json.load(fp)
    train_imgs = data['train']
    valid_imgs = data['val']

In [9]:
hue = {}
for valid in valid_imgs:
    for obj in valid['object']:
        if obj['name'] not in hue:
            hue[obj['name']] = 1
        else:
            hue[obj['name']] +=1
hue

{'bus': 184, 'motorbike': 150, 'car': 905, 'bicycle': 178}

In [10]:
hue = {}
for train in train_imgs:
    for obj in train['object']:
        if obj['name'] not in hue:
            hue[obj['name']] = 1
        else:
            hue[obj['name']] +=1
hue

{'car': 3537, 'bus': 699, 'bicycle': 659, 'motorbike': 651}

In [11]:
tr = Trainer(model=pretrained_model, input_size=608, max_box_per_image=25, labels=labels, anchors=[0.57273, 0.677385, 1.87446, 2.06253, 3.33843, 5.47434, 7.88282, 3.52778, 9.77052, 9.16828], have_head=True)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 608, 608, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 608, 608, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 608, 608, 32) 128         conv2d_1[0][0]                   
__________________________________________________________________________________________________
leaky_re_lu_1 (LeakyReLU)       (None, 608, 608, 32) 0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
max_poolin

In [12]:
!rm -rf wt
!mkdir wt
!rm -rf logs/log_ua_voc_4v2
!mkdir logs/log_ua_voc_4v2

In [13]:
# tr.randomize_weight()

In [14]:
tr.train(
    tensorboard_log='/src/workspace/logs/v2',
    saved_weights_name='./wt/v2_{epoch:02d}_{val_loss:.2f}.h5',
    train_imgs=train_imgs,
    batch_size=64,
    valid_imgs=valid_imgs,
    learning_rate=0.0008,
    valid_times=1,
    train_times=1,
    debug=False
)

Epoch 1/104
38/38 [==============================] - 89s 2s/step - loss: 467.9195 - val_loss: 7.2362

Epoch 00001: saving model to ./wt/ua_voc_4v2_01_7.24.h5
Epoch 2/104
38/38 [==============================] - 71s 2s/step - loss: 7.2352 - val_loss: 7.0546

Epoch 00002: saving model to ./wt/ua_voc_4v2_02_7.05.h5
Epoch 3/104
38/38 [==============================] - 77s 2s/step - loss: 7.0916 - val_loss: 6.9971

Epoch 00003: saving model to ./wt/ua_voc_4v2_03_7.00.h5
Epoch 4/104
38/38 [==============================] - 77s 2s/step - loss: 7.0431 - val_loss: 7.9355

Epoch 00004: saving model to ./wt/ua_voc_4v2_04_7.94.h5
Epoch 5/104
38/38 [==============================] - 76s 2s/step - loss: 10.0667 - val_loss: 7.9274

Epoch 00005: saving model to ./wt/ua_voc_4v2_05_7.93.h5
Epoch 6/104
38/38 [==============================] - 78s 2s/step - loss: 8.3895 - val_loss: 7.7855

Epoch 00006: saving model to ./wt/ua_voc_4v2_06_7.79.h5
Epoch 7/104
38/38 [==============================] - 78s 2s/s

In [15]:
from keras.models import save_model

In [16]:
# yolo_model_best = create_detector_model_from_trained(load_model('voc_4.h5', compile=False))
yolo_model_last = create_detector_model_from_trained(tr.model)

In [17]:
save_model(yolo_model_last, 'v2.h5', include_optimizer=False)

In [18]:
elapsed_time = time.time() - start_time
print(elapsed_time)

1854.9719166755676
